In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

# Загрузка данных
df = pd.read_csv('https://drive.google.com/uc?export=download&id=19X0ZP-Ff-2TAhag_HdzJYbM_oC_eqp2t', encoding='latin-1', usecols=[0, 1], names=['label', 'message'], skiprows=1)
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Предобработка текста
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])  # Удаление пунктуации
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Лемматизация и удаление стоп-слов
    return ' '.join(words)

df['processed_message'] = df['message'].apply(preprocess_text)

# Векторизация текста (используем Bag of Words и TF-IDF по очереди)
X_train, X_test, y_train, y_test = train_test_split(df['processed_message'], df['label'], test_size=0.2, random_state=42)

# Bag of Words
vectorizer_bow = CountVectorizer()
X_train_bow = vectorizer_bow.fit_transform(X_train)
X_test_bow = vectorizer_bow.transform(X_test)

# TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

# Оценка модели
model = LogisticRegression()
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"F1-score: {f1_score(y_test, y_pred)}")
    print(f"ROC-AUC: {roc_auc_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))

print("\nРезультаты с Bag of Words:")
evaluate_model(model, X_train_bow, X_test_bow, y_train, y_test)

print("\nРезультаты с TF-IDF:")
evaluate_model(model, X_train_tfidf, X_test_tfidf, y_train, y_test)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Результаты с Bag of Words:
Accuracy: 0.97847533632287
F1-score: 0.9130434782608695
ROC-AUC: 0.9199999999999999
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       1.00      0.84      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115


Результаты с TF-IDF:
Accuracy: 0.9506726457399103
F1-score: 0.782608695652174
ROC-AUC: 0.8279274611398965
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       965
           1       0.96      0.66      0.78       150

    accuracy                           0.95      1115
   macro avg       0.96      0.83      0.88      1115
weighted avg       0.95      0.95      0.95      1115




Bag of Words показал лучшую эффективность в классификации сообщений, достигнув 97.85% точности и 91.30% F1-score, в то время как TF-IDF показал 95.07% и 78.26% соответственно. Это связано с тем, что Bag of Words лучше сохраняет информацию о частоте ключевых слов, что критично для задач классификации, особенно в коротких текстах.